In [4]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv1D,Convolution2D, Bidirectional, LSTM, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import sklearn, json
import scipy.io as io
from typing import Any, Dict

Array = np.ndarray


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
dataset_path = '/mnt/Data/gmr/Dataset/'

In [3]:
with h5py.File(dataset_path +'X_train.mat', 'r') as f:
    X_train = np.array(f['X_train']).T
print(X_train.shape)
with h5py.File(dataset_path +'X_test.mat', 'r') as f:
    X_test = np.array(f['X_test']).T

(172500, 1024, 2)


In [4]:
lbl_train = io.loadmat(dataset_path +'lbl_train.mat')['lbl_train']
lbl_test = io.loadmat(dataset_path +'lbl_test.mat')['lbl_test']
print(lbl_test.shape)
print(lbl_train.shape)

(34500, 6)
(172500, 6)


In [5]:
print(lbl_train[0, :])

[ 1.00000000e+00 -8.00000000e+00  1.02400000e+03  1.44630639e-01
  1.23685330e-01  0.00000000e+00]


In [6]:
Y_train = io.loadmat(dataset_path +'Y_train.mat')
Y_train = Y_train['Y_train']
Y_train.shape

(172500, 23)

In [7]:
Y_test = io.loadmat(dataset_path +'Y_test.mat')
Y_test = Y_test['Y_test']
Y_test.shape

(34500, 23)

In [8]:
classes = ['LFM','2FSK','4FSK','8FSK', 'Costas','2PSK','4PSK','8PSK','Barker','Huffman','Frank','P1','P2','P3','P4','Px','Zadoff-Chu','T1','T2','T3','T4','NM','ruido']


In [9]:
len(classes)

23

In [10]:
I_x = X_train[:, :, 0]
Q_x = X_train[:, :, 1]
X_train[:,:,1] = np.arctan(Q_x, I_x)/ np.pi
X_train[:,:,0] = np.abs(I_x + 1j*Q_x)
I_t = X_test[:, :, 0]
Q_t = X_test[:, :, 1]
X_test[:, :, 0] = np.arctan(Q_t, I_t)/np.pi
X_test[:, :, 1] = np.abs(I_t + 1j*Q_t)

In [11]:
np.random.seed(2022)
X_train, Y_train, lbl_train = sklearn.utils.shuffle(X_train[:],Y_train[:], lbl_train[:], random_state=2022)
X_test, Y_test, lbl_test = sklearn.utils.shuffle(X_test[:], Y_test[:], lbl_test[:], random_state=2022)

In [12]:
def lstm_network(input_shape: int) -> Model:
    X_input = Input(input_shape)
    X = LSTM(128, return_sequences=True, name= 'lstm0')(X_input)
    X = LSTM(128, name='lstm1')(X)
    X = Dense(23, activation='softmax', name='fc0')(X)
    model = Model(inputs=X_input, outputs=X)
    model.summary()
    return model


In [13]:
model =lstm_network(X_train.shape[1:])

2022-07-06 15:49:13.832610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 15:49:13.881988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 15:49:13.882170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-06 15:49:13.882823: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 2)]         0         
                                                                 
 lstm0 (LSTM)                (None, 1024, 128)         67072     
                                                                 
 lstm1 (LSTM)                (None, 128)               131584    
                                                                 
 fc0 (Dense)                 (None, 23)                2967      
                                                                 
Total params: 201,623
Trainable params: 201,623
Non-trainable params: 0
_________________________________________________________________


In [14]:

c = [ModelCheckpoint(filepath='/mnt/Data/gmr/pruebas/bestmodel.h5', monitor='cal_loss', save_best_only=True)]
model.compile(optimizer=optimizers.Adam(10e-3), loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
path = '/mnt/Data/gmr/models/'

In [16]:
def Train(model: Model, data: Dict, n_epochs: int = 100, batch_size: int = 100):
    history = model.fit(data['X_train'], data['Y_train'], epochs = n_epochs, batch_size=batch_size, callbacks=c, validation_data=(data['X_test'], data['Y_test']))
    with open(path+'history_rnn.json', 'w') as f:
        json.dump(history.history, f)
    model_json = model.to_json()
    with open(path + 'model_rnn.json', 'w') as json_file:
        json_file.write(model_json)
def save_best():
    model.load_weights(path+'best_model.h5')
    with open(output_path + 'history_rnn.json', 'r') as f:
        history = json.load(f)
    

In [ ]:
Train(model, {'X_train': X_train, 'X_test': X_test, 'Y_train': Y_train, 'Y_test': Y_test})

2022-07-06 15:49:16.214338: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1413120000 exceeds 10% of free system memory.
2022-07-06 15:49:17.449332: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1413120000 exceeds 10% of free system memory.


Epoch 1/100


2022-07-06 15:49:20.872953: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


1725/1725 [==============================] - ETA: 0s - loss: 3.1381 - accuracy: 0.0447

2022-07-06 15:56:05.743540: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 282624000 exceeds 10% of free system memory.
2022-07-06 15:56:05.991525: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 282624000 exceeds 10% of free system memory.
